## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
#Setting up environment with initial libraries needed for importing, cleaning, and initial visualization
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import cufflinks as cf
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 25)
pd.set_option('display.float_format', '{:,}'.format)

sns.set_style('darkgrid')

%matplotlib inline

# Read the King County Housing csv into a dataframe with Pandas and assign it to the variable kchouse_df
kchouse_df = pd.read_csv('kc_house_data.csv')

# Make sure everything has been imported correctly and get a first look at the top five rows of data
kchouse_df.head()

C:\Users\jtang\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.0",3,1.0,1180,5650,1.0,nan,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,"538,000.0",3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,"1,991.0",98125,47.721000000000004,-122.319,1690,7639
2,5631500400,2/25/2015,"180,000.0",2,1.0,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,nan,98028,47.7379,-122.23299999999999,2720,8062
3,2487200875,12/9/2014,"604,000.0",4,3.0,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.39299999999999,1360,5000
4,1954400510,2/18/2015,"510,000.0",3,2.0,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [2]:
#print summary of dataframe
kchouse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [3]:
#call descriptive statistics of dataframe
kchouse_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,"21,597.0","21,597.0","21,597.0","21,597.0","21,597.0","21,597.0","21,597.0","19,221.0","21,534.0","21,597.0","21,597.0","21,597.0","21,597.0","17,755.0","21,597.0","21,597.0","21,597.0","21,597.0","21,597.0"
mean,"4,580,474,287.770987","540,296.5735055795",3.3731999814789093,2.1158262721674306,"2,080.3218502569803","15,099.408760475992",1.4940964022780943,0.007595858696217679,0.23386272870808952,3.4098254387183404,7.657915451220076,"1,788.5968421540028","1,970.9996758809093",83.6367783722895,"98,077.95184516368",47.56009299439737,-122.2139824975691,"1,986.6203176367087","12,758.283511598833"
std,"2,876,735,715.74778","367,368.1401013945",0.9262988945421479,0.7689842966527209,918.1061250800805,"41,412.636875502074",0.5396827909775457,0.08682484570054606,0.7656862011746727,0.6505456356725546,1.1731996637760254,827.7597611646754,29.375234132441644,399.9464138788162,53.51307235352816,0.13855176817307177,0.14072352882947373,685.2304719001586,"27,274.441950386343"
min,"1,000,102.0","78,000.0",1.0,0.5,370.0,520.0,1.0,0.0,0.0,1.0,3.0,370.0,"1,900.0",0.0,"98,001.0",47.1559,-122.51899999999999,399.0,651.0
25%,"2,123,049,175.0","322,000.0",3.0,1.75,"1,430.0","5,040.0",1.0,0.0,0.0,3.0,7.0,"1,190.0","1,951.0",0.0,"98,033.0",47.4711,-122.32799999999999,"1,490.0","5,100.0"
50%,"3,904,930,410.0","450,000.0",3.0,2.25,"1,910.0","7,618.0",1.5,0.0,0.0,3.0,7.0,"1,560.0","1,975.0",0.0,"98,065.0",47.5718,-122.23100000000001,"1,840.0","7,620.0"
75%,"7,308,900,490.0","645,000.0",4.0,2.5,"2,550.0","10,685.0",2.0,0.0,0.0,4.0,8.0,"2,210.0","1,997.0",0.0,"98,118.0",47.678000000000004,-122.125,"2,360.0","10,083.0"
max,"9,900,000,190.0","7,700,000.0",33.0,8.0,"13,540.0","1,651,359.0",3.5,1.0,4.0,5.0,13.0,"9,410.0","2,015.0","2,015.0","98,199.0",47.7776,-121.315,"6,210.0","871,200.0"


In [4]:
#print sum of missing values
kchouse_df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [16]:
#checking for duplicate rows across all columns

duplicate_rows = kchouse_df[kchouse_df.duplicated()]
print(f"The are {len(duplicate_rows)} duplicate rows in the dataframe.")

The are 0 duplicate rows in the dataframe.


In [13]:
#checking for duplicate values within each column

for col in kchouse_df:
    duplicate_id = kchouse_df[kchouse_df.duplicated(col)]
    print(f'There are {len(duplicate_id)} duplicate rows in {col}, ' 
          f'which represent {round(len(duplicate_id)/len(kchouse_df),3)}% of all entries in column.')

There are 177 duplicate rows in id, which represent 0.008% of all entries in column.
There are 21225 duplicate rows in date, which represent 0.983% of all entries in column.
There are 17975 duplicate rows in price, which represent 0.832% of all entries in column.
There are 21585 duplicate rows in bedrooms, which represent 0.999% of all entries in column.
There are 21568 duplicate rows in bathrooms, which represent 0.999% of all entries in column.
There are 20563 duplicate rows in sqft_living, which represent 0.952% of all entries in column.
There are 11821 duplicate rows in sqft_lot, which represent 0.547% of all entries in column.
There are 21591 duplicate rows in floors, which represent 1.0% of all entries in column.
There are 21594 duplicate rows in waterfront, which represent 1.0% of all entries in column.
There are 21591 duplicate rows in view, which represent 1.0% of all entries in column.
There are 21592 duplicate rows in condition, which represent 1.0% of all entries in column.

In [5]:
#convert date col object strings to datetime objects and call first 5 rows
kchouse_df['date'] = pd.to_datetime(kchouse_df.date, format='%m/%d/%Y')
kchouse_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,"221,900.0",3,1.0,1180,5650,1.0,nan,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,"538,000.0",3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,"1,991.0",98125,47.721000000000004,-122.319,1690,7639
2,5631500400,2015-02-25,"180,000.0",2,1.0,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,nan,98028,47.7379,-122.23299999999999,2720,8062
3,2487200875,2014-12-09,"604,000.0",4,3.0,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.39299999999999,1360,5000
4,1954400510,2015-02-18,"510,000.0",3,2.0,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [6]:
#use for loop to print descriptive statistics for all columns, including for object datatypes
for col in kchouse_df.columns:
    print(f'{col}')
    print(kchouse_df[col].describe())
    print('---------------------------')

id
count               21,597.0
mean    4,580,474,287.770987
std      2,876,735,715.74778
min              1,000,102.0
25%          2,123,049,175.0
50%          3,904,930,410.0
75%          7,308,900,490.0
max          9,900,000,190.0
Name: id, dtype: float64
---------------------------
date
count                   21597
unique                    372
top       2014-06-23 00:00:00
freq                      142
first     2014-05-02 00:00:00
last      2015-05-27 00:00:00
Name: date, dtype: object
---------------------------
price
count             21,597.0
mean    540,296.5735055795
std     367,368.1401013945
min               78,000.0
25%              322,000.0
50%              450,000.0
75%              645,000.0
max            7,700,000.0
Name: price, dtype: float64
---------------------------
bedrooms
count             21,597.0
mean    3.3731999814789093
std     0.9262988945421479
min                    1.0
25%                    3.0
50%                    3.0
75%                    4

In [7]:
for col in kchouse_df.columns:
    print(f'{col}')
    print(kchouse_df[col].describe())
    print('=========================================')

id
count               21,597.0
mean    4,580,474,287.770987
std      2,876,735,715.74778
min              1,000,102.0
25%          2,123,049,175.0
50%          3,904,930,410.0
75%          7,308,900,490.0
max          9,900,000,190.0
Name: id, dtype: float64
date
count                   21597
unique                    372
top       2014-06-23 00:00:00
freq                      142
first     2014-05-02 00:00:00
last      2015-05-27 00:00:00
Name: date, dtype: object
price
count             21,597.0
mean    540,296.5735055795
std     367,368.1401013945
min               78,000.0
25%              322,000.0
50%              450,000.0
75%              645,000.0
max            7,700,000.0
Name: price, dtype: float64
bedrooms
count             21,597.0
mean    3.3731999814789093
std     0.9262988945421479
min                    1.0
25%                    3.0
50%                    3.0
75%                    4.0
max                   33.0
Name: bedrooms, dtype: float64
bathrooms
count        

In [8]:
print(kchouse_df.median(), '\n\n')
print(kchouse_df.min(), '\n\n')
print(kchouse_df.max())

id                  3,904,930,410.0
price                     450,000.0
bedrooms                        3.0
bathrooms                      2.25
sqft_living                 1,910.0
sqft_lot                    7,618.0
floors                          1.5
waterfront                      0.0
view                            0.0
condition                       3.0
grade                           7.0
sqft_above                  1,560.0
yr_built                    1,975.0
yr_renovated                    0.0
zipcode                    98,065.0
lat                         47.5718
long            -122.23100000000001
sqft_living15               1,840.0
sqft_lot15                  7,620.0
dtype: float64 


id                           1000102
date             2014-05-02 00:00:00
price                       78,000.0
bedrooms                           1
bathrooms                        0.5
sqft_living                      370
sqft_lot                         520
floors                           1.0
wa

In [17]:
for col in kchouse_df.columns:
    print(col)
    print(kchouse_df[col].value_counts(bins=10))
    print('\n')

id
(990900110.8, 1980800119.6]     3052
(2970700128.4, 3960600137.2]    2981
(1980800119.6, 2970700128.4]    2859
(6930300163.6, 7920200172.4]    2495
(-8898898.089, 990900110.8]     2033
(7920200172.4, 8910100181.2]    1942
(8910100181.2, 9900000190.0]    1818
(5940400154.8, 6930300163.6]    1517
(4950500146.0, 5940400154.8]    1487
(3960600137.2, 4950500146.0]    1413
Name: id, dtype: int64


date
(2014-06-10, 2014-07-19]                       2847
(2014-07-19, 2014-08-27]                       2686
(2015-03-10, 2015-04-18]                       2601
(2014-05-01 14:38:23.999999999, 2014-06-10]    2424
(2014-10-05, 2014-11-13]                       2344
(2014-08-27, 2014-10-05]                       2143
(2014-11-13, 2014-12-22]                       1893
(2015-01-30, 2015-03-10]                       1743
(2015-04-18, 2015-05-27]                       1654
(2014-12-22, 2015-01-30]                       1262
Name: date, dtype: int64


price
(70377.999, 840200.0]     19053
(840200.0, 1

TypeError: bins argument only works with numeric data.

In [ ]:
#print descriptive statistics of sqft_basement column
kchouse_df['sqft_basement'].describe()

In [ ]:
#create new dataframe using groupby function on sqft_basement col and call groups in new dataframe
sqftbasegroup = kchouse_df.groupby('sqft_basement')
sqftbasegroup.groups

In [ ]:
#compute size of each group in groupby dataframe
sqftbasegroup.size()

In [ ]:
#call ? group in groupby dataframe
sqftbasegroup.get_group('?')

In [ ]:
#print descriptive statistics of ? group
sqftbasegroup.get_group('?').describe()

In [ ]:
#call row with to 33 bedrooms
kchouse_df.query('bedrooms == 33')

In [ ]:
#change 33 bedroom outlier to 3 bedrooms
kchouse_df.bedrooms[kchouse_df.bedrooms == 33] = 3

In [ ]:
#call updated property id
kchouse_df.query('id == 2402100895')

In [ ]:
kchouse_df.query('bathrooms == .5')

In [ ]:
kchouse_df.query('condition > 3' and 'grade > 6').head()

In [ ]:
#kchouse_sqftbase_df = kchouse_df['sqft_basement'].str.replace('?',)

In [ ]:
kczipcity_df = pd.read_csv('kc_city_zip.csv')
kczipcity_df.head()

In [ ]:
kchousecity_df = pd.merge(left=kchouse_df, right=kczipcity_df, how='left', left_on='zipcode', right_on='zipcode')
kchousecity_df.head()

In [ ]:
kchousecity_group = kchousecity_df.groupby('city')

In [ ]:
kchousecity_group.get_group('Auburn').describe()

In [ ]:
kchousecity_df.groupby('city')['price'].head()

In [ ]:
for city in kchousecity_group.groups:
    print(f'{city}')
    print(kchousecity_group.get_group(city))

kchousecity_group.get_group('Woodinville')

In [27]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols

outcome = 'price'

x_cols = kchousecity_df.drop(['price', 'date','sqft_basement', 'id'], axis=1).columns
predictors = '+'.join(x_cols)

f = outcome + '~' + predictors

model = smf.ols(formula=f, data=kchousecity_df).fit()
model.summary2()

NameError: name 'kchousecity_df' is not defined

In [26]:
pearsoncorr = kchousecity_df.corr('pearson')

NameError: name 'kchousecity_df' is not defined

In [25]:
plt.figure(figsize=(25,25))
sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

NameError: name 'pearsoncorr' is not defined

<Figure size 1800x1800 with 0 Axes>

#checking for duplicate values within each column
for col in kchouse_df:
    duplicate_id = kchouse_df[kchouse_df.duplicated(col)]
    print(f'There are {len(duplicate_id)} duplicate rows in {col}, ' 
          f'which represent {round(len(duplicate_id)/len(kchouse_df) * 100, 2)}% of all entries in column.')

#checking for the sum of missing values within each column
for col in kchouse_df:
    mv = kchouse_df[col].isna().sum()
    if mv > 0:
        print(f'{col} column has {mv} missing values')

In [18]:
kchouse_df2 = pd.read_csv('kc_house_data.csv')

In [23]:
kchouse_df2.replace('?', 0)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.0",3,1.0,1180,5650,1.0,nan,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,"538,000.0",3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,"1,991.0",98125,47.721000000000004,-122.319,1690,7639
2,5631500400,2/25/2015,"180,000.0",2,1.0,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,nan,98028,47.7379,-122.23299999999999,2720,8062
3,2487200875,12/9/2014,"604,000.0",4,3.0,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.39299999999999,1360,5000
4,1954400510,2/18/2015,"510,000.0",3,2.0,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,"360,000.0",3,2.5,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,"400,000.0",4,2.5,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,2014,0.0,98146,47.5107,-122.36200000000001,1830,7200
21594,1523300141,6/23/2014,"402,101.0",2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,2009,0.0,98144,47.5944,-122.29899999999999,1020,2007
21595,291310100,1/16/2015,"400,000.0",3,2.5,1600,2388,2.0,nan,0.0,3,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [24]:
sqft_basegroup2 = kchouse_df2.groupby('sqft_basement')
sqft_basegroup2.get_group('?')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
6,1321400060,6/27/2014,"257,500.0",3,2.25,1715,6819,2.0,0.0,0.0,3,7,1715,?,1995,0.0,98003,47.3097,-122.32700000000001,2238,6819
18,16000397,12/5/2014,"189,000.0",2,1.0,1200,9850,1.0,0.0,0.0,4,7,1200,?,1921,0.0,98002,47.3089,-122.21,1060,5095
42,7203220400,7/7/2014,"861,990.0",5,2.75,3595,5639,2.0,0.0,0.0,3,9,3595,?,2014,0.0,98053,47.6848,-122.016,3625,5639
79,1531000030,3/23/2015,"720,000.0",4,2.5,3450,39683,2.0,0.0,0.0,3,10,3450,?,2002,0.0,98010,47.342,-122.025,3350,39750
112,2525310310,9/16/2014,"272,500.0",3,1.75,1540,12600,1.0,0.0,0.0,4,7,1160,?,1980,0.0,98038,47.3624,-122.031,1540,11656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21442,3226049565,7/11/2014,"504,600.0",5,3.0,2360,5000,1.0,0.0,0.0,3,7,1390,?,2008,0.0,98103,47.6931,-122.33,2180,5009
21447,1760650900,7/21/2014,"337,500.0",4,2.5,2330,4907,2.0,0.0,0.0,3,7,2330,?,2013,nan,98042,47.358999999999995,-122.081,2300,3836
21473,6021503707,1/20/2015,"352,500.0",2,2.5,980,1010,3.0,nan,0.0,3,8,980,?,2008,0.0,98117,47.6844,-122.387,980,1023
21519,2909310100,10/15/2014,"332,000.0",4,2.5,2380,5737,2.0,0.0,0.0,3,7,2380,?,2010,0.0,98023,47.2815,-122.35600000000001,2380,5396
